In [ ]:
import os
import mlflow
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load environment variables
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MODEL_NAME = os.getenv("MODEL_NAME")

# Set MinIO credentials for artifact upload
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID", "minioadmin")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY", "minioadmin")
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL", "http://localhost:9000")

In [ ]:
print(f"MLflow Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"Model Name: {MODEL_NAME}")
print(f"MinIO Endpoint: {os.environ.get('MLFLOW_S3_ENDPOINT_URL')}")

In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("iris_classifier_demo")

In [ ]:
# Load the Iris dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
with mlflow.start_run(run_name="iris_classifier_demo") as run:
    # Initialize the RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Log the model and metrics
    mlflow.log_params(clf.get_params())
    mlflow.log_metrics({"accuracy": accuracy})

    # Log the model with input example to avoid warning
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="model",
        input_example=X_train[:5],  # Add input example
        registered_model_name=MODEL_NAME  # Register directly when logging
    )
    
    print(f"Model logged and registered as '{MODEL_NAME}'")
    print(f"Run ID: {run.info.run_id}")
    print(f"Artifacts stored in MinIO at: {run.info.artifact_uri}")